In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

In [2]:
data_train = pd.read_csv("./datasets/titanic/train.csv")
data_test = pd.read_csv("./datasets/titanic/test.csv")

In [3]:
def createTitle(df):
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
    return df

In [4]:
def titleMapping(df):
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    df['Title'] = df['Title'].map(title_mapping)
    df['Title'] = df['Title'].fillna(0)
    
    return df

In [5]:
def dropColumns(df):
    df = df.drop(['Name','PassengerId'], axis=1)
    df = df.drop(['Ticket', 'Cabin'], axis=1)
    return df

In [6]:
def sexMapping(df):
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    
    return df

In [7]:
def setAge(df):
    df.dropna(subset=['Age'], inplace=True)
    df['Age'] = df['Age'].astype(int)

    df.loc[df['Age'] <= 16, 'Age'] = 0
    df['Age'] = df.loc[(df['Age'] > 16) & df['Age'] <= 32, 'Age'] = 1
    df['Age'] = df.loc[(df['Age'] > 32) & df['Age'] <= 48, 'Age'] = 2
    df['Age'] = df.loc[(df['Age'] > 48) & df['Age'] <= 64, 'Age'] = 3
    df['Age'] = df.loc[df['Age'] > 64, 'Age'] = 4
    
    return df

In [8]:
def createAgeClass(df):
    df['Age_Class'] = df['Age']*df['Pclass']
    
    return df

In [9]:
def setEmbarked(df):
    df['Embarked'] = df['Embarked'].fillna('S')
    df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q':2} ).astype(int)
    
    return df

In [10]:
def setFare(df):
    df['Fare'] = df['Fare'].dropna()
    df.loc[df['Fare'] <= 7.91, 'Fare'] = 0
    df['Fare'] = df.loc[(df['Fare'] > 7.91) & df['Fare'] <= 14.454, 'Fare'] = 1
    df['Fare'] = df.loc[(df['Fare'] > 14.454) & df['Fare'] <= 31, 'Fare'] = 2
    df['Fare'] = df.loc[df['Fare'] > 31, 'Fare'] = 3
    
    return df

In [11]:
def dataPipeline(df):
    df = createTitle(df)
    df = titleMapping(df)
    df = dropColumns(df)
    df = sexMapping(df)
    df = setAge(df)
    df = createAgeClass(df)
    df = setEmbarked(df)
    df = setFare(df)
    
    return df

In [12]:
data_train = dataPipeline(data_train)
data_test = dataPipeline(data_test)

In [19]:
data_train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Age_Class
0,0,3,0,4,1,0,3,0,1,12
1,1,1,1,4,1,0,3,1,3,4
2,1,3,1,4,0,0,3,0,2,12
3,1,1,1,4,1,0,3,0,3,4
4,0,3,0,4,0,0,3,0,1,12
6,0,1,0,4,0,0,3,0,1,4
7,0,3,0,4,3,1,3,0,4,12
8,1,3,1,4,0,2,3,0,3,12
9,1,2,1,4,1,0,3,1,3,8
10,1,3,1,4,1,1,3,0,2,12


In [15]:
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [16]:
train, test = train_test_split(data_train, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

456 train examples
115 validation examples
143 test examples


In [22]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Survived')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds


In [23]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False)
test_ds = df_to_dataset(test, shuffle=False)

In [25]:
data_train.columns.tolist()

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'Title',
 'Age_Class']

In [27]:
feature_columns = []

# numeric cols
for header in ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'Age_Class']:
  feature_columns.append(feature_column.numeric_column(header))

In [28]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [56]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [60]:
model = tf.keras.Sequential([
  feature_layer,
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Train for 15 steps, validate for 4 steps
Epoch 1/100
15/15 [==============================] - 1s 39ms/step - loss: 0.6947 - accuracy: 0.5702 - val_loss: 0.6886 - val_accuracy: 0.7043
Epoch 2/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6600 - accuracy: 0.6996 - val_loss: 0.6717 - val_accuracy: 0.7130
Epoch 3/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6463 - accuracy: 0.7478 - val_loss: 0.6744 - val_accuracy: 0.6957
Epoch 4/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6448 - accuracy: 0.7697 - val_loss: 0.6581 - val_accuracy: 0.7043
Epoch 5/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6336 - accuracy: 0.7851 - val_loss: 0.6518 - val_accuracy: 0.7652
Epoch 6/100
15/15 [==============================] - 0s 5ms/step - loss: 0.6342 - accuracy: 0.7961 - val_loss: 0.6507 - val_accuracy: 0.7217
Epoch 7/100
15/15 [==============================] - 0s 5ms/step - loss: 0.6277 - accuracy: 0.8048 - val_loss: 0

15/15 [==============================] - 0s 5ms/step - loss: 0.6152 - accuracy: 0.8268 - val_loss: 0.6454 - val_accuracy: 0.7565
Epoch 59/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6135 - accuracy: 0.8443 - val_loss: 0.6472 - val_accuracy: 0.7652
Epoch 60/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6138 - accuracy: 0.8399 - val_loss: 0.6472 - val_accuracy: 0.7478
Epoch 61/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6075 - accuracy: 0.8311 - val_loss: 0.6484 - val_accuracy: 0.7565
Epoch 62/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6062 - accuracy: 0.8487 - val_loss: 0.6436 - val_accuracy: 0.8087
Epoch 63/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6177 - accuracy: 0.8355 - val_loss: 0.6479 - val_accuracy: 0.7478
Epoch 64/100
15/15 [==============================] - 0s 5ms/step - loss: 0.6031 - accuracy: 0.8377 - val_loss: 0.6493 - val_accuracy: 0.7565
Epoch 65/100
15/15 